In [49]:
import os

cwd = os.getcwd()

In [50]:
import pandas as pd

df = pd.read_csv(cwd + '/data/processed/posts_train_test_clean.csv')

In [51]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [52]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

## 100 Tags mais usadas

In [53]:
def calculateTagsEntries(df):
    count = {}
    
    for tags in df['targets']:
        for tag in tags:
            if(tag not in count.keys()):
                dataTag = {
                    'entries': 1,
                }
                count[tag] = dataTag
            else:
                count[tag]['entries'] += 1
    return count

def sortTags(count):
    return dict(sorted(count.items(), key=lambda item: -item[1]['entries']))

In [54]:
classes = []

def createClassesColumn(df):
    for documentTags in df['targets']:
        wasInDocumentTags = False

        for documentTag in documentTags:
            if(documentTag in MostUsedTags):
                wasInDocumentTags = True
                classes.append(documentTag)
                break
        if(not wasInDocumentTags):
            classes.append('unknown-tag')
    df['class'] = classes    

In [55]:
MostUsedTags = list(sortTags(calculateTagsEntries(df)).keys())[:100]
createClassesColumn(df)

In [56]:
y = df['class']

In [57]:
df

,Unnamed: 0,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text,targets,class
0,69943,16835963,Sidekiq not processing queue,"<p>What possible reasons can <a href=""https://...",<ruby-on-rails-3><redis><sidekiq>,2013-05-30 12:19:00,2013-05-30 12:53:32,16836794,16836794,71,sidekiq not processing queue what possible rea...,sidekiq process queue possibl reason sidekiq p...,"[ruby-on-rails-3, redis, sidekiq]",unknown-tag
1,79655,31667160,Running docker container : iptables: No chain/...,<p>I'm trying to run a container but I get the...,<docker><port><iptables>,2015-07-28 04:03:53,2015-07-28 04:14:04,31667244,31667244,65,running docker container iptables no chain tar...,run docker contain iptabl chain target match n...,"[docker, port, iptables]",docker
2,97858,3299648,Python Compilation/Interpretation Process,<p>I'm trying to understand the python compile...,<python><compiler-construction><interpreter><p...,2010-07-21 13:21:47,2010-07-21 13:28:54,3299724,3299724,53,python compilation interpretation process i m ...,python compil interpret process tri understand...,"[python, compiler-construction, interpreter, p...",python
3,14078,3664272,Is std::vector so much slower than plain arrays?,<p>I've always thought it's the general wisdom...,<c++><arrays><performance><stl><vector>,2010-09-08 02:38:41,2010-09-08 02:57:40,3664349,3664349,239,is std vector so much slower than plain arrays...,std vector much slower plain array alway thoug...,"[c++, arrays, performance, stl, vector]",c++
4,46299,13342123,How to get relative path of a file in visual s...,<p>I am trying to get the path of an image fil...,<c#><visual-studio>,2012-11-12 10:33:32,2013-10-11 10:57:06,19316470,19316470,100,how to get relative path of a file in visual s...,get rel path file visual studio tri get path i...,"[c#, visual-studio]",c#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73917,91379,1563062,Prevent form submission with enter key,<p>I just wrote this nifty little function whi...,<javascript><jquery><html>,2009-10-13 21:39:53,2009-10-14 14:20:48,1566586,1566586,57,prevent form submission with enter key i just ...,prevent form submiss enter key wrote nifti lit...,"[javascript, jquery, html]",javascript
73918,96168,57202043,Windows Defender might be impacting your build...,<p>After I updated my PyCharm IDE to 19.2.0 fr...,<android-studio><jetbrains-ide>,2019-07-25 12:31:51,2019-10-17 13:22:37,58433724,58433724,54,windows defender might be impacting your build...,window defend might impact build perform updat...,"[android-studio, jetbrains-ide]",android-studio
73919,21235,1030043,HTML table headers always visible at top of wi...,"<p>I would like to be able to ""tweak"" an HTML ...",<html><html-table>,2009-06-23 00:17:12,2013-10-02 16:27:04,19141921,19141921,179,html table headers always visible at top of wi...,html tabl header alway visibl top window view ...,"[html, html-table]",html
73920,10891,15826745,TypeScript: Creating an empty typed container ...,"<p>I am creating simple logic game called ""Thr...",<javascript><arrays><typescript>,2013-04-05 05:43:23,2017-04-05 12:10:42,43230812,43230812,286,typescript creating an empty typed container a...,typescript creat empti type contain array crea...,"[javascript, arrays, typescript]",javascript


In [58]:
targets_set = set(MostUsedTags + ['unknown-tag'])
print(len(targets_set))

101


In [59]:
words_post = df.complete_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=20)
# vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
X = vectorizer.transform(words_post)

In [60]:
import gc

gc.collect()

0

In [61]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier(random_state=44, strategy="prior")
dummy_model.fit(X_train, y_train)

DummyClassifier(random_state=44)

In [63]:
y_pred = dummy_model.predict(X_test)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                     precision    recall  f1-score   support

               .net       0.00      0.00      0.00       127
               ajax       0.00      0.00      0.00         4
          algorithm       0.00      0.00      0.00        51
amazon-web-services       0.00      0.00      0.00        28
            android       0.00      0.00      0.00       801
     android-studio       0.00      0.00      0.00        16
            angular       0.00      0.00      0.00       116
          angularjs       0.00      0.00      0.00        65
             arrays       0.00      0.00      0.00        31
            asp.net       0.00      0.00      0.00        67
        asp.net-mvc       0.00      0.00      0.00        51
               bash       0.00      0.00      0.00       138
                  c       0.00      0.00      0.00       194
                 c#      

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
from joblib import dump, load
dump(dummy_model, 'models/dummy_model_less_tags.joblib') 

['models/dummy_model_less_tags.joblib']

In [66]:
dummy_model.classes_

array(['.net', 'ajax', 'algorithm', 'amazon-web-services', 'android',
       'android-studio', 'angular', 'angularjs', 'arrays', 'asp.net',
       'asp.net-mvc', 'bash', 'c', 'c#', 'c++', 'c++11', 'class',
       'cocoa-touch', 'command-line', 'css', 'dart', 'database',
       'dataframe', 'date', 'datetime', 'debugging', 'dictionary',
       'django', 'docker', 'eclipse', 'entity-framework', 'exception',
       'file', 'flutter', 'function', 'git', 'github', 'go',
       'google-chrome', 'gradle', 'html', 'http', 'image',
       'intellij-idea', 'ios', 'iphone', 'java', 'java-8', 'javascript',
       'jquery', 'json', 'laravel', 'linq', 'linux', 'list', 'macos',
       'matplotlib', 'maven', 'mongodb', 'multithreading', 'mysql',
       'node.js', 'npm', 'numpy', 'objective-c', 'oop', 'pandas',
       'performance', 'php', 'postgresql', 'python', 'python-3.x', 'r',
       'reactjs', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'scala',
       'shell', 'sorting', 'spring', 'sql', 'sql-serv

In [67]:
dummy_model.n_classes_

101

In [68]:
dummy_model.class_prior_

array([7.82927778e-03, 5.58026278e-04, 3.38197744e-03, 2.80704128e-03,
       5.33168744e-02, 1.13296244e-03, 8.03219643e-03, 4.78549808e-03,
       2.11373590e-03, 4.87004752e-03, 4.19365203e-03, 9.58790605e-03,
       1.25809561e-02, 7.75656526e-02, 3.98904239e-02, 1.18369210e-04,
       2.36738421e-04, 3.21287857e-04, 8.45494361e-04, 1.53034479e-02,
       2.21519522e-03, 2.58721274e-03, 1.01459323e-04, 2.36738421e-04,
       2.53648308e-04, 8.79314135e-04, 4.73476842e-04, 4.95459695e-03,
       5.54644301e-03, 3.48343677e-03, 8.11674586e-04, 2.36738421e-04,
       8.45494361e-04, 2.92541049e-03, 6.93305376e-04, 2.90850060e-02,
       2.11373590e-03, 4.56566955e-03, 1.60643929e-03, 8.79314135e-04,
       2.28114378e-02, 3.17905880e-03, 7.94764699e-04, 2.18137545e-03,
       2.64639735e-02, 7.38962071e-03, 8.49721832e-02, 1.86008759e-04,
       9.78575173e-02, 1.31220725e-02, 3.02686981e-03, 1.33588109e-03,
       4.73476842e-04, 1.16340024e-02, 6.93305376e-04, 3.90618395e-03,
      

In [69]:
dummy_model.n_outputs_

1